In [ ]:
import psycopg2

conn = psycopg2.connect(host='...',
                        database='...',
                        user='...',
                        password='...')

cur = conn.cursor()

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
import pandas as pd
import numpy as np 

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [ ]:
# postgresql로 데이터 불러오기
cur.execute("SELECT * FROM customer")
customer = pd.DataFrame(data = cur.fetchall(), columns=["Id",	"Gender",	"Type of Travel",	"Age", "Customer Type",	"Class",	"Flight Distance",	"Departure Delay in Minutes",	"Arrival Delay in Minutes"])
cur.execute("SELECT * FROM satisfaction")
satisfaction =  pd.DataFrame(data = cur.fetchall(), columns=["Id",	"Seat comfort",	"Departure/Arrival time convenient",	"Food and drink",	"Gate location",	"Inflight wifi service",	"Inflight entertainment",	"Online support",	"Ease of Online booking",	"On-board service",	"Leg room service",	"Baggage handling",	"Checkin service",	"Cleanliness",	"Online boarding"])
satisfaction.drop('Id', inplace=True, axis=1)
cur.execute("SELECT * FROM customer_satisfaction")
target = pd.DataFrame(data = cur.fetchall(), columns=['Id', 'satisfaction'])
cur.close()
conn.close()
# features와 target 구분
X = pd.concat([customer, satisfaction], axis=1)
y = target['satisfaction']

In [ ]:
X

,Id,Gender,Type of Travel,Age,Customer Type,Class,Flight Distance,Departure Delay in Minutes,Arrival Delay in Minutes,Seat comfort,...,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding
0,0,Male,Personal Travel,59,Loyal Customer,Eco,1170,58,53.0,3,...,2,3,2,2,5,3,2,1,5,2
1,1,Female,Business travel,41,Loyal Customer,Business,3882,0,0.0,2,...,5,4,3,2,2,2,2,3,2,4
2,2,Female,Business travel,48,Loyal Customer,Business,3349,52,23.0,1,...,4,4,4,5,5,4,5,5,5,4
3,3,Male,Business travel,41,Loyal Customer,Business,3866,11,9.0,4,...,4,5,5,4,4,4,4,4,4,3
4,4,Male,Business travel,57,Loyal Customer,Business,877,0,0.0,4,...,5,5,4,5,5,4,5,5,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29995,Male,Personal Travel,10,Loyal Customer,Business,2253,113,112.0,3,...,4,3,4,4,1,5,4,1,3,4
29996,29996,Male,Business travel,45,Loyal Customer,Eco Plus,2804,0,0.0,2,...,2,2,2,2,1,5,2,3,1,2
29997,29997,Female,Business travel,48,Loyal Customer,Business,2348,10,0.0,1,...,5,5,5,5,5,5,5,3,5,4
29998,29998,Male,Business travel,20,disloyal Customer,Eco,1123,0,7.0,2,...,1,2,1,1,4,4,5,4,5,1


In [ ]:
y

0            0
1            1
2            2
3            3
4            4
         ...  
29995    29995
29996    29996
29997    29997
29998    29998
29999    29999
Name: satisfaction, Length: 30000, dtype: int64

In [ ]:
X.columns

Index(['Id', 'Gender', 'Type of Travel', 'Age', 'Customer Type', 'Class',
       'Flight Distance', 'Departure Delay in Minutes',
       'Arrival Delay in Minutes', 'Seat comfort',
       'Departure/Arrival time convenient', 'Food and drink', 'Gate location',
       'Inflight wifi service', 'Inflight entertainment', 'Online support',
       'Ease of Online booking', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Cleanliness',
       'Online boarding'],
      dtype='object')

In [ ]:
# 불필요 feature & 특정 feature와 상관관계 있는 feature 삭제 
X.drop(['Id', 'Departure Delay in Minutes'], axis=1, inplace=True)

In [ ]:
X.head()

,Gender,Type of Travel,Age,Customer Type,Class,Flight Distance,Arrival Delay in Minutes,Seat comfort,Departure/Arrival time convenient,Food and drink,...,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding
0,Male,Personal Travel,59,Loyal Customer,Eco,1170,53.0,3,5,3,...,2,3,2,2,5,3,2,1,5,2
1,Female,Business travel,41,Loyal Customer,Business,3882,0.0,2,1,1,...,5,4,3,2,2,2,2,3,2,4
2,Female,Business travel,48,Loyal Customer,Business,3349,23.0,1,4,1,...,4,4,4,5,5,4,5,5,5,4
3,Male,Business travel,41,Loyal Customer,Business,3866,9.0,4,4,4,...,4,5,5,4,4,4,4,4,4,3
4,Male,Business travel,57,Loyal Customer,Business,877,0.0,4,4,4,...,5,5,4,5,5,4,5,5,5,4


In [ ]:
print('Gender :', X['Gender'].unique())
print('Type of travel :', X['Type of Travel'].unique())
print('Class ;', X['Class'].unique())
print('Customer type :', X['Customer Type'].unique())

Gender : ['Male' 'Female']
Type of travel : ['Personal Travel' 'Business travel']
Class ; ['Eco' 'Business' 'Eco Plus']
Customer type : ['Loyal Customer' 'disloyal Customer']


In [ ]:
def transform(df):
  df['Gender'].replace(['Male', 'Female'], [1,0], inplace=True)
  df['Type of Travel'].replace(['Personal Travel', 'Business travel'], [1, 0], inplace=True)
  df['Class'].replace(['Business', 'Eco', 'Eco Plus'], [3, 2, 1], inplace=True)
  df['Customer Type'].replace(['Loyal Customer', 'disloyal Customer'], [0, 1], inplace=True)
  return df

transform(X).head()

,Gender,Type of Travel,Age,Customer Type,Class,Flight Distance,Arrival Delay in Minutes,Seat comfort,Departure/Arrival time convenient,Food and drink,...,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding
0,1,1,59,0,2,1170,53.0,3,5,3,...,2,3,2,2,5,3,2,1,5,2
1,0,0,41,0,3,3882,0.0,2,1,1,...,5,4,3,2,2,2,2,3,2,4
2,0,0,48,0,3,3349,23.0,1,4,1,...,4,4,4,5,5,4,5,5,5,4
3,1,0,41,0,3,3866,9.0,4,4,4,...,4,5,5,4,4,4,4,4,4,3
4,1,0,57,0,3,877,0.0,4,4,4,...,5,5,4,5,5,4,5,5,5,4


In [ ]:
X_train = X[:24000]
X_test = X[24000:]
y_train = y[:24000]
y_test = y[24000:]
# 훈련데이터, 테스트데이터 분리 확인
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(24000, 21) (24000,)
(6000, 21) (6000,)


In [ ]:
y_train

0            0
1            1
2            2
3            3
4            4
         ...  
23995    23995
23996    23996
23997    23997
23998    23998
23999    23999
Name: satisfaction, Length: 24000, dtype: int64

In [ ]:
X_train.tail()

,Gender,Type of Travel,Age,Customer Type,Class,Flight Distance,Arrival Delay in Minutes,Seat comfort,Departure/Arrival time convenient,Food and drink,...,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding
23995,1,0,66,0,3,3702,0.0,2,2,4,...,3,2,1,5,5,5,5,3,5,4
23996,1,0,34,0,3,264,20.0,1,1,1,...,3,5,5,3,3,4,3,1,3,4
23997,1,0,31,0,3,5337,0.0,1,1,1,...,5,5,5,5,2,5,2,3,1,5
23998,1,0,27,1,2,2664,96.0,2,2,2,...,5,2,5,5,1,5,5,5,2,5
23999,1,1,51,0,1,2540,0.0,1,2,1,...,4,1,4,4,3,1,2,3,2,4


In [ ]:
X_test.tail()

,Gender,Type of Travel,Age,Customer Type,Class,Flight Distance,Arrival Delay in Minutes,Seat comfort,Departure/Arrival time convenient,Food and drink,...,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding
29995,1,1,10,0,3,2253,112.0,3,4,3,...,4,3,4,4,1,5,4,1,3,4
29996,1,0,45,0,1,2804,0.0,2,2,3,...,2,2,2,2,1,5,2,3,1,2
29997,0,0,48,0,3,2348,0.0,1,1,1,...,5,5,5,5,5,5,5,3,5,4
29998,1,0,20,1,2,1123,7.0,2,0,2,...,1,2,1,1,4,4,5,4,5,1
29999,0,1,58,0,2,434,0.0,5,5,2,...,5,5,5,5,4,4,3,3,5,5


In [ ]:
# pip install category_encoders

In [ ]:
pipe = make_pipeline(
    SimpleImputer(),
    RandomForestClassifier(n_jobs=-1)
)

pipe

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('randomforestclassifier', RandomForestClassifier(n_jobs=-1))])

In [ ]:
pipe.fit(X_train, y_train)
y_test_predict = pipe.predict(X_test)

In [ ]:
print(classification_report(y_test, y_test_predict))

NameError: ignored

In [ ]:
y_train_predict = pipe.predict(X_train)
print(classification_report(y_train, y_train_predict))

In [ ]:
y_train_pred = pd.DataFrame({'satisfaction_predicted': y_train_predict})
y_train_pred

In [ ]:
y_test_pred = pd.DataFrame({'satisfaction_predicted': y_test_predict})
y_test_pred

In [ ]:
y_pred = pd.concat([y_train_pred, y_test_pred], axis=0).reset_index(drop=True)
y_pred = y_pred.reset_index().rename({"index":"id"}, axis=1)
y_pred

In [ ]:
y_pred.to_csv('./prediction.csv', index=False)

In [ ]:
import pickle

with open('model.pkl', 'wb') as pickle_file:
  pickle.dump(pipe, pickle_file)